<h1><center>

![](https://i.imgur.com/jrRGMfx.png)

AI PRACTITIONER COURSE 

ASSIGNMENT - SESSION 19 - Attention and Pretrained Language Model

**&copy; 2020 VTCA-COTAI. Internal Use Only.**

</center></h1>

**DATASET LINK** https://drive.google.com/drive/folders/1-3Al7UX_I0gbtufXaXa5U3gqLncuUwFC

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

# Chuẩn bị dữ liệu

In [ ]:
from tqdm.notebook import tqdm
import re

**Cài đặt thư viện ``transformers``**

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 5.4MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
     |████████████████████████████████| 2.9MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=eaedece588f368994c2b822c822020ef6aea71177ae98accb5ff6ec231096b68
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


**Đường dẫn đến thư mục chứa data**

In [ ]:
# path to data
train_path = '/content/drive/MyDrive/sentiment /train.crash'
test_path = '/content/drive/MyDrive/sentiment /test.crash'

**Khởi tạo Tokenizer từ pretrained model của Transformer**

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
test_token = 'Tôi là sinh viên trường Đại Học Công Nghệ. tao là ai?'
print(tokenizer.tokenize(test_token))
print(tokenizer.encode(test_token))

['Tôi', 'là', 'sinh', 'viên', 'trường', 'Đại', 'Học', 'Công', 'Nghệ@@', '.', 'tao', 'là', 'ai@@', '?']
[0, 218, 8, 418, 1430, 212, 3449, 5271, 3344, 58460, 5, 6377, 8, 6010, 114, 2]


Trong đó **index** ``0, 1, 2, 3`` là 4 ký tự đặc biệt

In [ ]:
print(tokenizer.convert_ids_to_tokens(0))
print(tokenizer.convert_ids_to_tokens(1))
print(tokenizer.convert_ids_to_tokens(2))
print(tokenizer.convert_ids_to_tokens(3))

<s>
<pad>
</s>
<unk>


Ví dụ về từ ghép

In [ ]:
test_token = 'sinh_viên'
tokenizer.encode(test_token)

[0, 649, 2]

**Lấy dữ liệu Train**

In [ ]:
train_text, train_masks, train_ids, train_labels = [], [], [], []

with open(train_path, 'r') as f_r:
  data = f_r.read().strip()

data = re.findall('train_[\s\S]+?\"\n[01]\n\n', data)
print('Clean completed')

for sample in tqdm(data):
  splits = sample.strip().split('\n')

  id = splits[0]
  label = int(splits[-1])
  text = ' '.join(splits[1:-1])[1:-1]
  train_text.append(text)
  text = tokenizer.encode_plus( text,
                                max_length=150,
                                return_token_type_ids=False,
                                padding=True,
                                return_attention_mask=True,
                                return_tensors='np',  # Return Numpy array
                              )

  train_masks.append(text['attention_mask'])
  train_ids.append(text['input_ids'])
  train_labels.append(label)

FileNotFoundError: ignored

**Xem thử dữ liệu Train**

In [ ]:
for i in range(10):
  print(train_text[i])
  print('Positive' if train_labels[i] == 0 else 'Negative')
  print()

**Lấy 1 phần dữ liệu Train làm Validation**

In [ ]:
val_ids = train_ids[10000:]
train_ids = train_ids[:10000]

val_masks = train_masks[10000:]
train_masks = train_masks[:10000]

val_labels = train_labels[10000:]
train_labels = train_labels[:10000]

**Lấy dữ liệu Test**

In [ ]:
test_masks, test_ids = [], []

with open(test_path, 'r') as f_r:
  data = f_r.read().strip()

data = re.findall('test_[\s\S]+?\"\n\n', data)
print('Clean completed')

for sample in tqdm(data):
  splits = sample.strip().split('\n')

  id = splits[0]
  text = ' '.join(splits[1:])[1:-1]
  text = tokenizer.encode_plus( text,
                                max_length=150,
                                return_token_type_ids=False,
                                padding=True,
                                return_attention_mask=True,
                                return_tensors='np',  # Return PyTorch tensors
                              )
  test_masks.append(text['attention_mask'])
  test_ids.append(text['input_ids'])

**Xem thử dữ liệu Train sau khi encode**

In [ ]:
# print out 1st train sample
print(train_ids[0])
print()
print(train_masks[0])

**Chuyển đổi từ numpy array sang Torch Tensor**

Lưu ý ``train_ids`` hiện đang có shape là ``10000, 1, 150`` ta cần chuyển chúng sang ``10000, 150`` bằng hàm ``torch.squeeze`` (tương đương với ``np.squeeze``)

In [ ]:
import torch

train_ids = torch.squeeze(torch.tensor(train_ids))
train_masks = torch.squeeze(torch.tensor(train_masks))
train_labels = torch.tensor(train_labels)

val_ids = torch.squeeze(torch.tensor(val_ids))
val_masks = torch.squeeze(torch.tensor(val_masks))
val_labels = torch.tensor(val_labels)

**Khởi tạo Dataset bằng framework PyTorch**

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

train_data = TensorDataset(train_ids, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_ids, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
validation_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

**Khởi tạo pretrained model**

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
    "vinai/phobert-base", from_tf=False, num_labels = 2, output_hidden_states=False,
)
model = RobertaForSequenceClassification.from_pretrained(
    "vinai/phobert-base",
    config=config
)
model.cuda()

**Các hàm hỗ trợ**

In [ ]:
import time
import datetime

# Hàm tính thời gian
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Bắt đầu quá trình Train

**ĐỪNG SỢ**

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
from transformers import get_linear_schedule_with_warmup

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

device = torch.device('cuda')
epochs = 1
# Measure the total training time for the whole run.
total_t0 = time.time()
total_steps = len(train_dataloader) * epochs
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)



# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        out = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += out.loss.item()

        # Perform a backward pass to calculate the gradients.
        out.loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            out = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += out.loss

        # Move logits and labels to CPU
        logits = out.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

**Lưu lại trọng số đã học**

In [ ]:
torch.save(model.state_dict(), '/content/model')

**Vẽ Learning Curve**

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats



import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

# Inference

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def encode(sentence):
  encode_sent = tokenizer.encode_plus(sentence, 
                                      max_length=150, 
                                      return_token_type_ids=False,
                                      pad_to_max_length=True,
                                      return_attention_mask=True,
                                      return_tensors='pt' )
  sent_id = encode_sent['input_ids'].to('cuda')
  sent_mask = encode_sent['attention_mask'].to('cuda')
  return sent_id, sent_mask

In [ ]:
def output(model, text):
  input_ids, attention_mask = encode(text)
  logits = model(input_ids,attention_mask).logits
  prediction = torch.argmax(logits, dim=1)
  if prediction == torch.tensor(0):
      print('Positive')
  else:
      print('Negative')

In [ ]:
BERT_SA = RobertaForSequenceClassification.from_pretrained("/content/model",config = config)
BERT_SA.cuda()
BERT_SA.eval()
text = "muốn ói"
output(BERT_SA, text)